<a href="https://colab.research.google.com/github/phu-minh/CS116.M11.KHTN/blob/main/CS116_Week4_PCA_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML
from base64 import b64encode
import os
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
def changeVideoCodec(input_path, output_path): 
    bashCommand = f"ffmpeg -i {input_path} -y {output_path}"
    import subprocess
    process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()

def displayVideo(path):
    video = open(path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(video).decode()
    return HTML("""
    <video width=1600 controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

def collectFrames(video_path):
    frame_list = []
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    if cap.isOpened():
        frame = None
        while True:
            try:
                is_success, frame = cap.read()
            except cv2.error:
                continue

            if not is_success:
                break

            # convert cv2.COLOR_BGR2GRAY
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frame_list.append(image)
        if frame:
          cv2_imshow(frame)    
    cap.release()
    return frame_list,width,height

def compressAndRestoreImgPCA(frame_list, n_components = 64):
    pca_frame_list = []
    my_pca = PCA(n_components)

    for frame in frame_list:
        img = frame
        #print(img.shape)
        my_pca.fit(img)

        pca_img = my_pca.transform(img)
        #print(pca_img.shape)

        temp = my_pca.inverse_transform(pca_img)
        temp = temp.astype(np.uint8)
        #cv2_imshow(temp)
        pca_frame_list.append(temp) 
    return pca_frame_list 

def convertFramesToVideo(frame_list, output_path,fps = 30):
    height, width = frame_list[0].shape
    size = (width, height)

    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'DIVX'), fps, size, 0)
    
    for i in range(len(frame_list)):
        # writing to a image list
        out.write(frame_list[i])

    out.release()

###Play video

In [ ]:
save_path = '/content/drive/MyDrive/CS116/video.mp4'
compressed_path = '/content/video.mp4'

In [ ]:
changeVideoCodec(save_path,compressed_path)

In [ ]:
displayVideo(compressed_path)

###PCA Cách 1: PCA mỗi frame riêng biệt

In [ ]:
frame_list,width,height = collectFrames(save_path)

In [ ]:
print(len(frame_list))

600


In [ ]:
print(width,height)

640 358


In [ ]:
pca_frame_list = compressAndRestoreImgPCA(frame_list,24)

In [ ]:
len(pca_frame_list)

600

In [ ]:
res_avi_path = '/content/pca1.avi'
res_path = '/content/pca1.mp4'

In [ ]:
convertFramesToVideo(pca_frame_list, res_avi_path)

In [ ]:
changeVideoCodec(res_avi_path,res_path)

In [ ]:
displayVideo(res_path)

###PCA Cách 2

In [ ]:
frame_list,width,height = collectFrames(save_path)

In [ ]:
cube = frame_list[0].flatten()
for frame in frame_list:
    frame_flattened = frame.flatten()
    cube = np.vstack((cube, frame_flattened))
cube = np.delete(cube, 0, 0)


In [ ]:
pca = PCA(n_components=32)
compressed_cube = pca.fit_transform(cube)
print(pca.explained_variance_ratio_.sum())

0.8469159909297065


In [ ]:
decoded_cube = pca.inverse_transform(compressed_cube)
h, w = frame_list[0].shape
new_frame_list = []
for i in range(len(decoded_cube)):
  new_frame_list.append(decoded_cube[i, :].reshape(h, w))
new_frame_list = [frame.astype("uint8") for frame in new_frame_list]

In [ ]:
res_avi_2_path = '/content/2.avi'
res_2_path = '/content/2.mp4'

In [ ]:
convertFramesToVideo(new_frame_list, res_avi_2_path)
changeVideoCodec(res_avi_2_path, res_2_path)
displayVideo(res_2_path)